In [ ]:
import langextract as lx
import json
import pandas as pd


In [ ]:
#Example 1 — straightforward outpatient note

example = "A 45-year-old man visited the Outpatient department complaining of persistent lower back pain that had lasted for three weeks." \
"He was prescribed ibuprofen 400 mg taken orally twice a day to relieve the discomfort."

#🩺 Example 2 — surgical ward note

example1 = "A 62-year-old woman was admitted to the Surgery department for postoperative wound infection that had been present for five days./
"The physician started her on cefazolin 1 g intravenous every 8 hours for infection control."

In [ ]:

# Define extraction prompt
prompt_description = (
    "Extract patient age, department, problem, How long problem Occured, medication, dosage, route, frequency. Use exact text for extractions. "
    "Do not paraphrase or overlap entities. Do not mix up the examples with input text provided. If you cannot find an entity, leave it blank." \
    "Make sure to only extract what is explicitly mentioned in the text. Make sure to not infer any information that is not directly stated in the text." \
    "Provide the extractions in the order specified. Do not duplicate extractions even if there are multiple answers for an entity just choose one." \
    "Emit each unique span once. If you already output the exact same text for a class, do not repeat it."
)


# Define example data with entities in order of appearance
examples = [
    lx.data.ExampleData(
        exampletext= example,
        extractions=[
            lx.data.Extraction(extraction_class="patient age", extraction_text="45-year-old"),
            lx.data.Extraction(extraction_class="department", extraction_text="Outpatient department"),
            lx.data.Extraction(extraction_class="problem", extraction_text="complaining of persistent lower back pain"),
            lx.data.Extraction(extraction_class="How long problem Occured", extraction_text="for three weeks"),
            lx.data.Extraction(extraction_class="medication", extraction_text="ibuprofen"),
            lx.data.Extraction(extraction_class="dosage", extraction_text="400 mg"),
            lx.data.Extraction(extraction_class="route", extraction_text="orally"),
            lx.data.Extraction(extraction_class="frequency", extraction_text="twice a day"),

        ]
    )
]




In [ ]:
# get result using ollaama model llama3.2:latest already download

# increase ollama api timeout
input_text = example

result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt_description,
    examples=examples,
    model_id="gemma2:2b",
        model_url="http://localhost:11434",
    fence_output=False,
    use_schema_constraints=False,     extraction_passes=1,    
    max_workers=20,         # Parallel processing for speed
                language_model_params={
                "keep_alive": 10*60,   # 10 minutes
                "timeout": 10*60       # 10 minutes
            }
)


e:\LLM\Extract Clinical Notes Information\venv\Lib\site-packages\langextract\extraction.py:186: UserWarning: batch_length (10) < max_workers (20). Only 10 workers will be used. Set batch_length >= max_workers for optimal parallelization.
  warnings.warn(


In [ ]:
# Display entities with positions
print(f"Input: {input_text}\n")
print("Extracted entities:")
for entity in result.extractions:
    position_info = ""
    if entity.char_interval:
        start, end = entity.char_interval.start_pos, entity.char_interval.end_pos
        position_info = f" (pos: {start}-{end})"
    print(f"• {entity.extraction_class.capitalize()}: {entity.extraction_text}{position_info}")



Input: A a sixteen year-old girl, presented to our Outpatient department with the complaints of discomfort in the neck and lower back as well as restriction of body movements. She was not able to maintain an erect posture and would tend to fall on either side while standing up from a sitting position. She would keep her head turned to the right and upwards due to the sustained contraction of the neck muscles. There was a sideways bending of the back in the lumbar region. To counter the abnormal positioning of the back and neck, she would keep her limbs in a specific position to allow her body weight to be supported. Due to the restrictions with the body movements at the neck and in the lumbar region, she would require assistance in standing and walking. She would require her parents to help her with daily chores, including all activities of self-care.
She had been experiencing these difficulties for the past four months since when she was introduced to olanzapine tablets for the contro

In [ ]:
# Try on another example to test model results
input_text1 = example1

print(input_text1)

This is the case of a 56-year-old man that was complaining of a dump pain on the right back and a swelling right in this place for several weeks. The patient was in good state and very active. There was not any health problem in the past except a thoracic trauma at work one year ago. In that time the patient was diagnosed with a simple fracture of the 9th right rib without any other consequences.
On the X-ray was seen a shadow in the lower part of the right hemithorax. After that, it was decided to perform a CT-scan of the thorax that revealed a tumor of the thoracic wall in the right hemithorax that measured 8 × 4 cm and had a heterogeneous density inside of it. The tumor had involved and destructed the 9th rib and was lying even in two adjacent intercostal spaces, but without involving the lung and muscular layers. On lung window of the CT-scan were seen micronodular infiltrations of both lungs with diameters up to 5 mm and only one nodule in lower lobe of the right lung with diamete

In [36]:
result1 = lx.extract(
    text_or_documents=input_text1,
    prompt_description=prompt_description,
    examples=examples,
    model_id="gemma2:2b",
        model_url="http://localhost:11434",
    fence_output=False,
    use_schema_constraints=False,     extraction_passes=1,    
                language_model_params={
                "keep_alive": 10*60,   # 10 minutes
                "timeout": 10*60       # 10 minutes
            }
)

In [37]:
# Display entities with positions
print(f"Input: {input_text1}\n")
print("Extracted entities:")
for entity in result1.extractions:
    position_info = ""
    if entity.char_interval:
        start, end = entity.char_interval.start_pos, entity.char_interval.end_pos
        position_info = f" (pos: {start}-{end})"
    print(f"• {entity.extraction_class.capitalize()}: {entity.extraction_text}{position_info}")



Input: This is the case of a 56-year-old man that was complaining of a dump pain on the right back and a swelling right in this place for several weeks. The patient was in good state and very active. There was not any health problem in the past except a thoracic trauma at work one year ago. In that time the patient was diagnosed with a simple fracture of the 9th right rib without any other consequences.
On the X-ray was seen a shadow in the lower part of the right hemithorax. After that, it was decided to perform a CT-scan of the thorax that revealed a tumor of the thoracic wall in the right hemithorax that measured 8 × 4 cm and had a heterogeneous density inside of it. The tumor had involved and destructed the 9th rib and was lying even in two adjacent intercostal spaces, but without involving the lung and muscular layers. On lung window of the CT-scan were seen micronodular infiltrations of both lungs with diameters up to 5 mm and only one nodule in lower lobe of the right lung with 